In [ ]:
# Imports

import torch
from torch import nn
import pytorch_lightning as pl

In [ ]:
# Data

class RandomDataModule(pl.LightningDataModule):
    def __init__(self, n_segments=1000, n_timesteps=100, n_features=8, batch_size=32):
        super().__init__()
        self.n_segments = n_segments
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = torch.randn(self.n_segments, self.n_timesteps, self.n_features)
        self.val_dataset = torch.randn(self.n_segments, self.n_timesteps, self.n_features)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size)

In [ ]:
# Model

class Autoencoder(pl.LightningModule):
    def __init__(self, n_features=8, hidden_size=8):
        super().__init__()

        self.hidden_size = hidden_size

        self.encoder = nn.LSTM(n_features, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, n_features, batch_first=True)

    def forward(self, x):
        batch_size = x.size(0)
        n_timesteps = x.size(1)
        y_enc, (h_enc, c_enc) = self.encoder(x)
        x_enc = torch.rand(batch_size, n_timesteps, self.hidden_size)
        y_dec, (h_dec, c_dec) = self.decoder(x_enc, (h_enc, c_enc))
        return y_dec

    def training_step(self, batch, batch_idx):
        X = batch
        X_reconn = self(X)
        loss = nn.functional.mse_loss(X_reconn, X)
        self.log('train/loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X = batch
        X_reconn = self(X)
        loss = nn.functional.mse_loss(X_reconn, X)
        self.log('val/loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-2)

In [ ]:
# Experiment

n_segments = 1000
n_timesteps = 120*10  # 10 sec
n_features = 59
hidden_size = 59

datamodule = RandomDataModule(n_segments, n_timesteps, n_features)

model = Autoencoder(n_features=n_features, hidden_size=hidden_size)

trainer = pl.Trainer(max_epochs=1000,accelerator='auto', log_every_n_steps=1)

trainer.fit(model, datamodule=datamodule)